## Libraries install and loading

In [ ]:
# install.packages("openssl")
# install.packages("arsenal")
# install.packages("reshape")
# install.packages("ggplot2")
# install.packages("ggnewscale")

In [ ]:
library(ggplot2)
library(ggnewscale)

## Data load

In [ ]:
password <- ""
key <- charToRaw(password)

In [ ]:
url <- "https://gitlab.com/MNCDS/CDS/esercitazione_accesso_dati/-/raw/main/data/encrypted_tabUU.rds?ref_type=heads"
download.file(url, "encrypted_tabUU.rds", mode="wb")
encrypted_tabUU <- readRDS("encrypted_tabUU.rds")
tabUU <- unserialize(openssl::aes_cbc_decrypt(encrypted_tabUU, key))
data.table::setDT(tabUU)
head(tabUU)

In [ ]:
url <- "https://gitlab.com/MNCDS/CDS/esercitazione_accesso_dati/-/raw/main/data/encrypted_tabUM_RIC.rds?ref_type=heads"
download.file(url, "encrypted_tabUM_RIC.rds", mode="wb")
encrypted_tabUM_RIC <- readRDS("encrypted_tabUM_RIC.rds")
tabUM_RIC <- unserialize(openssl::aes_cbc_decrypt(encrypted_tabUM_RIC, key))
data.table::setDT(tabUM_RIC)
head(tabUM_RIC)

In [ ]:
url <- "https://gitlab.com/MNCDS/CDS/esercitazione_accesso_dati/-/raw/main/data/encrypted_Dictionary.rds?ref_type=heads"
download.file(url, "encrypted_Dictionary.rds", mode="wb")
encrypted_Dictionary <- readRDS("encrypted_Dictionary.rds")
Dictionary <- unserialize(openssl::aes_cbc_decrypt(encrypted_Dictionary, key))
data.table::setDT(Dictionary)
head(Dictionary)

## Tabella UU: esplorazione

In [ ]:
colnames(tabUU)

In [ ]:
cols_tabUU <- data.frame(variable = colnames(tabUU))
cols_tabUU <- merge(
    cols_tabUU,
    Dictionary[, c("variable", "label", "definition")],
    by = "variable",
    all.x = TRUE
)

head(cols_tabUU)

### Tabella UU: descrittiva (1)

In [ ]:
# Define variables of interest
vars_descriptive <- c(
    "age", "genderClear", "admReas", "typeStatus_RIC", "typeTrauma", "Infections", "sofa", "saps", 
    "sevInfectionsSepsis3", "icuSevInfections_RIC", "comorbidities_immunosuppression_RIC", "numFailure_admissionValueCat", "icuMortality", "hospOutcome_RIC"
)

# Define grouping variable
var_group <- "Anno"

# Define subset and rename columns for better output readability
tabUU_descriptive <- tabUU[, c(..vars_descriptive, ..var_group)]
labels_descriptive <- Dictionary$label[match(vars_descriptive, Dictionary$variable)]
labels_descriptive[is.na(labels_descriptive)] <- vars_descriptive[is.na(labels_descriptive)]
data.table::setnames(tabUU_descriptive, old = vars_descriptive, new = labels_descriptive)

# Describe
fml <- as.formula(paste(var_group, "~ ."))
table_d1 <- arsenal::tableby(
    fml,
    data = tabUU_descriptive,
    numeric.test = 'kwt',
    numeric.stats = c( 'meansd', 'medianq1q3', 'Nmiss' ),
    selectall.stats = c("countpct","Nmiss"),
    cat.stats = c("countpct","Nmiss"),
    digits = 0,
    conf.level = 0.95,
    total.pos = 'before',
    stats.labels = list( 'Nmiss' = 'Missing' ),
    cat.simplify = FALSE
)

summary(table_d1, text = TRUE)

## Tabella UM_RIC: esplorazione

In [ ]:
colnames(tabUM_RIC)

In [ ]:
cols_tabUM_RIC <- data.frame(variable = colnames(tabUM_RIC))
cols_tabUM_RIC <- merge(
    cols_tabUM_RIC,
    Dictionary[, c("variable", "label", "definition")],
    by = "variable",
    all.x = TRUE
)

head(cols_tabUM_RIC)

## Analisi 

### Tabella UU: descrittiva (2)

Tabella UU arricchita con informazioni su infezioni all'ammissione

In [ ]:
# Define list of columns with variables of interest
columns_fungi <- c(
    "albicans_RIC",
    "auris_RIC",
    "glabrata_RIC",
    "krusei_RIC",
    "otherMushrooms_RIC",
    "parapsilosis_RIC",
    "tropicalis_RIC",
    "undefinedCandida_RIC",
    "otherCandida_RIC",
    "pneumocystisJirovecii_RIC"
)

column_aspergillus <- "aspergillus_RIC"

In [ ]:
# Keep only rows for infections at admission to ICU in tabUM_RIC
tabUM_RIC_admission <- tabUM_RIC[Infezione == "AMM"]

# Create summary column True/False 
# True if at least one column in columns_fungi is (1)
# False if all columns in columns_fungi are (0)
tabUM_RIC_admission[
    ,
    ohter_fungi := apply(.SD, 1, function(x) as.integer(any(x == "1"))),
    .SDcols = columns_fungi
]

In [ ]:
# Group by year, centreCode, admissionKey
# A single block for combination (year, centreCode, admissionKey) is created
# If at least one row has aspergillus_RIC == "1"
# -> microorganism "Aspergillo"
# !!! Cases with both "Aspergillo" and other fungi are labelled in the same way as "Aspergillo"

table_micro_year <- tabUM_RIC_admission[, .(
    microorganism_adm = data.table::fifelse(
        any(aspergillus_RIC == "1"), 
        "Aspergillo",
        data.table::fifelse(
            any(ohter_fungi == 1), 
            "Altri funghi",
            data.table::fifelse(
                any(num_micro > 0),
                "Altro",
                NA_character_
            )
        )
    )
), by = .(admissionKey)]

table(table_micro_year$microorganism_adm)

In [ ]:
# If we want to distinguish cases where aspergillus is the only present fungus
# we need to add another fifelse

table_micro_year <- tabUM_RIC_admission[, .(
    microorganism_adm = data.table::fifelse(
        any(aspergillus_RIC == "1") & any(ohter_fungi == 1), 
        "Aspergillo + altro fungo",
        data.table::fifelse(
            any(aspergillus_RIC == "1"),
            "Aspergillo",
            data.table::fifelse(
                any(ohter_fungi == 1), 
                "Altri funghi",
                data.table::fifelse(
                    any(num_micro > 0),
                    "Altro",
                    NA_character_
                )
            )
        )
    )
), by = .(admissionKey)]

table(table_micro_year$microorganism_adm)

In [ ]:
# We can just keep the information in this table or
# 
# We can add it to our main table with one row per patient

if (!("microorganism_adm" %in% colnames(tabUU))) {
    tabUU <- merge(
        tabUU,
        table_micro_year,
        by = c("admissionKey"),
        all.x = TRUE  
    ) 
}

table(tabUU$microorganism_adm, exclude = NULL)

In [ ]:
# Define variables of interest
vars_descriptive <- c(
    "age", "genderClear", "admReas", "typeStatus_RIC", "typeTrauma", "Infections", "sofa", "saps", 
    "sevInfectionsSepsis3", "icuSevInfections_RIC", "comorbidities_immunosuppression_RIC", "numFailure_admissionValueCat", "icuMortality", "hospOutcome_RIC"
)

# Define grouping variable
var_group <- "microorganism_adm"

# Define subset and rename columns for better output readability
tabUU_descriptive <- tabUU[, c(..vars_descriptive, ..var_group)]
labels_descriptive <- Dictionary$label[match(vars_descriptive, Dictionary$variable)]
labels_descriptive[is.na(labels_descriptive)] <- vars_descriptive[is.na(labels_descriptive)]
data.table::setnames(tabUU_descriptive, old = vars_descriptive, new = labels_descriptive)

# Describe
fml <- as.formula(paste(var_group, "~ ."))
table_d2 <- arsenal::tableby(
    fml,
    data = tabUU_descriptive,
    numeric.test = 'kwt',
    numeric.stats = c( 'meansd', 'medianq1q3', 'Nmiss' ),
    selectall.stats = c("countpct","Nmiss"),
    cat.stats = c("countpct","Nmiss"),
    digits = 0,
    conf.level = 0.95,
    total.pos = 'before',
    stats.labels = list( 'Nmiss' = 'Missing' ),
    cat.simplify = FALSE
)

summary(table_d2, text = TRUE)

### Tabella UM: Numero di infezioni, per anno, per microorganismo

In [ ]:
columns_candida <- c(
    "albicans_RIC", "auris_RIC", "glabrata_RIC", "krusei_RIC", "parapsilosis_RIC", "tropicalis_RIC", 
    "undefinedCandida_RIC", "otherCandida_RIC"
)

tabUM_RIC[
    ,
    fungi := apply(.SD, 1, function(x) as.integer(any(x == "1"))),
    .SDcols = c(columns_fungi, columns_candida, column_aspergillus)
]

tabUM_RIC[
    ,
    candida := apply(.SD, 1, function(x) as.integer(any(x == "1"))),
    .SDcols = columns_candida
]                   

tabUM_RIC[
    ,
    aspergillus := apply(.SD, 1, function(x) as.integer(any(x == "1"))),
    .SDcols = column_aspergillus
]
                   
table_num_inf <- tabUM_RIC[, .(
    n_infections       = sum(num_micro > 0),
    n_infections_fungi = sum(as.numeric(fungi == TRUE), na.rm = TRUE),
    n_infections_asper = sum(as.numeric(aspergillus == TRUE), na.rm = TRUE),
    n_infections_candi = sum(as.numeric(candida == TRUE), na.rm = TRUE)
), by = Anno]

table_num_inf[, `:=`(
    perc_fungi_over_infections = round(100*n_infections_fungi/n_infections, 2),
    perc_asper_over_infections = round(100*n_infections_asper/n_infections, 2),
    perc_asper_over_fungi = round(100*n_infections_asper/n_infections_fungi, 2),
    perc_candi_over_infections = round(100*n_infections_candi/n_infections, 2),
    perc_candi_over_fungi = round(100*n_infections_candi/n_infections_fungi, 2)
)]

table_num_inf

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5) 

plot_n_asper_over_infections <- ggplot(
    data = table_num_inf,
    aes(x = factor(Anno), y = n_infections_asper, fill = perc_asper_over_infections)
) + 
geom_col() +
geom_text(
    aes(label = paste0(round(perc_asper_over_infections, 2), "%")),
    vjust = -0.5, 
    size = 5
) +
scale_fill_gradient(
    name = "Percentuale (%)",
    low = "goldenrod", high = "darkred"
) +
ylim(0, max(table_num_inf$n_infections_asper, na.rm = TRUE) * 1.2) +
labs(
    title = "Infezioni da aspergillo",
    subtitle = "Percentuale su totale infezioni per anno",
    x = "", 
    y = "Numero infezioni da aspergillo",
    fill = "Percentuale (%)"
) +
theme_minimal(base_size = 16) +
theme(
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    panel.border = element_rect(color = "black", fill = NA, linewidth = 1),
    plot.title = element_text(face = "bold", hjust = 0.5),
    plot.subtitle = element_text(hjust = 0.5)
)

plot_n_asper_over_infections

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 5) 

plot_n_fungi_perc_asper <- ggplot(
    data = table_num_inf,
    aes(x = Anno)
) + 
geom_col(
    aes(
        y = n_infections_fungi, fill = "Infezioni fungine"
    ),
    alpha = 0.5
) + 
scale_fill_manual(name = NULL, values =  c("Infezioni fungine" = "grey80")) +
ggnewscale::new_scale_fill() +
geom_col(aes(y = n_infections_asper, fill = perc_asper_over_fungi)) +
scale_fill_gradient(
    name = "Infezioni da aspergillo (%)",
    low = "goldenrod", high = "darkred"
) +
scale_x_continuous(breaks = table_num_inf$Anno) + 
geom_text(
    aes(y = n_infections_asper, label = paste0(round(perc_asper_over_fungi, 2), "%")),
    vjust = -0.5, 
    size = 5
) +
labs(
    title = "Infezioni fungine",
    subtitle = "Percentuale di infezioni da aspergillo su infezioni fungine",
    x = "", 
    y = "Numero infezioni",
    fill = "Infezioni da aspergillo (%)"
) +
theme_minimal(base_size = 16) +
theme(
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA),
    panel.border = element_rect(color = "black", fill = NA, linewidth = 1),
    plot.title = element_text(face = "bold", hjust = 0.5),
    plot.subtitle = element_text(hjust = 0.5),
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
)

plot_n_fungi_perc_asper

### Tabella UM: Numero di infezioni, per anno, per microorganismo

In [ ]:
tabUM_RIC$Infezione[tabUM_RIC$Infezione %in% "NEW"] <- "DEG"

table_num_inf_amm_deg <- tabUM_RIC[, .(
    n_infections_ad       = sum(num_micro > 0),
    n_infections_fungi_ad = sum(as.numeric(fungi == TRUE), na.rm = TRUE),
    n_infections_asper_ad = sum(as.numeric(aspergillus == TRUE), na.rm = TRUE),
    n_infections_candi_ad = sum(as.numeric(candida == TRUE), na.rm = TRUE)
), by = .(Anno, Infezione)]

table_num_inf_amm_deg <- merge(
    table_num_inf_amm_deg,
    table_num_inf,
    by = "Anno",
    all.x = TRUE
)

table_num_inf_amm_deg[, `:=`(
    perc_asper_over_fungi_ad = round(100*n_infections_asper_ad/n_infections_asper, 2),
    Infezione = factor(Infezione, levels = c("AMM", "DEG"))
)]

head(table_num_inf_amm_deg)

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 7) 

plot_perc_asper_amm_deg <- ggplot(
    data = table_num_inf_amm_deg,
    aes(x = Anno, y = perc_asper_over_fungi, fill = Infezione)
) +
geom_col(width = 1, color = "white", position = "stack") +
geom_text(
    aes(label = paste0(perc_asper_over_fungi_ad, "%")),
    position = position_stack(vjust = 0.6),
    size = 4,
    color = "white",
    # fontface = "bold"
) +
coord_polar(theta = "x", start = 0) +
scale_x_continuous(breaks = table_num_inf_amm_deg$Anno) + 
scale_fill_manual(
    values = c("AMM" = "goldenrod", "DEG" = "darkred"),
    labels = c("Ammissione", "Degenza")
) +
labs(
    title = "Infezioni da aspergillo",
    subtitle = "Periodo di insorgenza infezione",
    fill = "",
    x = NULL,
    y = NULL
) +
theme_minimal(base_size = 16) +
theme(
    axis.text.y = element_blank(),
    axis.text.x = element_text(face = "bold", size = 14),
    axis.ticks = element_blank(),    
    panel.grid = element_blank(),
    legend.position = "bottom",
    plot.title = element_text(face = "bold", hjust = 0.5),
    plot.subtitle = element_text(hjust = 0.5),
    plot.background = element_rect(fill = "white", color = NA),
    panel.background = element_rect(fill = "white", color = NA)
)

plot_perc_asper_amm_deg